<a href="https://colab.research.google.com/github/mahtabkarami/Projects/blob/main/Brain_tumor_detection_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# نصب کتابخانه‌های لازم
!pip install flask pyngrok kagglehub

# ایمپورت‌ها
import kagglehub
from flask import Flask, request, render_template_string, send_from_directory
from pyngrok import ngrok
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers, losses
import numpy as np
import os
import shutil

# دانلود دیتاست
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
print("Path to dataset files:", path)

# مسیر دیتا
train_data_dir = os.path.join(path, 'Training')
test_data_dir = os.path.join(path, 'Testing')

# دیتاجنریتور
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir, target_size=(64, 64),
    color_mode='grayscale', class_mode='categorical',
    batch_size=128, shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir, target_size=(64, 64),
    color_mode='grayscale', class_mode='categorical',
    batch_size=128, shuffle=False
)

# تعداد کلاس‌ها
num_classes = train_generator.num_classes

# ساخت مدل
myInput = layers.Input(shape=(64, 64, 1))
conv1 = layers.Conv2D(32, 3, padding='same', strides=2)(myInput)
bn1 = layers.BatchNormalization()(conv1)
ac1 = layers.Activation('relu')(bn1)
conv2 = layers.Conv2D(64, 3, padding='same', strides=2)(ac1)
bn2 = layers.BatchNormalization()(conv2)
ac2 = layers.Activation('relu')(bn2)
flat = layers.Flatten()(ac2)
out_layer = layers.Dense(num_classes, activation='softmax')(flat)

myModel = models.Model(myInput, out_layer)
myModel.compile(optimizer=optimizers.SGD(learning_rate=0.001),
                loss=losses.categorical_crossentropy,
                metrics=['accuracy'])

# آموزش مدل
history = myModel.fit(train_generator, epochs=10, validation_data=test_generator)

# ذخیره و بارگذاری مدل
model_path = "/content/my_mri_model.h5"
myModel.save(model_path)
model = load_model(model_path)

# کلاس‌ها
class_labels = list(train_generator.class_indices.keys())

# پوشه آپلود
upload_folder = '/content/uploads'
os.makedirs(upload_folder, exist_ok=True)

# ساخت اپ Flask
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = upload_folder

html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Brain Tumor MRI Prediction</title>
</head>
<body style="text-align:center; font-family: Arial;">
    <h1>Upload an MRI Image</h1>
    <form method="POST" action="/" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*" required>
        <br><br>
        <input type="submit" value="Predict">
    </form>

    {% if image_url %}
        <h2>Uploaded Image:</h2>
        <img src="{{ image_url }}" style="width:300px; height:300px; object-fit:cover; border:2px solid black;">
        <h2>Predicted Tumor Type: {{ prediction }}</h2>
        <h3>Confidence: {{ confidence }}</h3>
    {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def upload_predict():
    prediction = None
    confidence = None
    image_url = None
    if request.method == "POST":
        file = request.files["file"]
        if file:
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
            file.save(filepath)

            img = image.load_img(filepath, target_size=(64, 64), color_mode='grayscale')
            img_array = image.img_to_array(img)
            img_array = img_array / 255.0
            img_array = np.expand_dims(img_array, axis=0)

            pred = model.predict(img_array)
            pred_class = np.argmax(pred)
            pred_confidence = np.max(pred)

            prediction = class_labels[pred_class]
            confidence = f"{pred_confidence * 100:.2f}%"
            image_url = f"/uploads/{file.filename}"

    return render_template_string(html_template, prediction=prediction, confidence=confidence, image_url=image_url)

@app.route("/uploads/<filename>")
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__ == "__main__":
    os.system("ngrok authtoken 2wIkinplPcxwWDOumn7c2cFJYts_3fwGHsZW4gAByMoAUixQQ")
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)
    app.run(port=5000, host="0.0.0.0")
    app.run(port=5000, host="0.0.0.0", debug=False, use_reloader=False)

Path to dataset files: /kaggle/input/brain-tumor-mri-dataset
Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 24s 510ms/step - accuracy: 0.4392 - loss: 1.2500 - val_accuracy: 0.2288 - val_loss: 1.3790
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 40s 485ms/step - accuracy: 0.7027 - loss: 0.7963 - val_accuracy: 0.2288 - val_loss: 1.4197
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 24s 527ms/step - accuracy: 0.7444 - loss: 0.6788 - val_accuracy: 0.2288 - val_loss: 1.4582
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 21s 467ms/step - accuracy: 0.7780 - loss: 0.6051 - val_accuracy: 0.2288 - val_loss: 1.5306
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 22s 489ms/step - accuracy: 0.7959 - loss: 0.5658 - val_accuracy: 0.2349 - val_loss: 1.5038
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 22s 486ms/step - accuracy: 0.8129 - loss: 0.5276 - val_accuracy: 0.2525 - val_loss: 1.5198
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 23s 522ms/step - accuracy: 0.8265 - loss: 0

Public URL: NgrokTunnel: "https://d6bf-34-86-102-217.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 07:25:32] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 07:25:32] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 07:29:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 07:29:04] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 07:30:17] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Apr/2025 07:30:17] "GET /uploads/Tr-no_0031.jpg HTTP/1.1" 200 -
